In [1]:
import os

import meshio
import numpy as np
from fimpy.solver import FIMPY

from sklearn.neighbors import BallTree
from numba import njit, prange
from numba.typed import List

Import of Cupy failed. The GPU version of fimpy will be unavailble. Message: No module named 'cupy'


## Load a Mesh 
This can be any mesh format that meshio can read. However, it needs to contain linear tetrahedral elements only.

In [2]:
# File to read
mesh_name = os.path.join("control_arm", "control_arm.msh")

# Read mesh with meshio
mesh = meshio.read(mesh_name)
points = mesh.points
elements = mesh.cells[0].data

## Set boundary conditions 
`injection_locs` is a list of 3D coordinates on the surface of the domain and prescribes the injection gate locations. It is used to compute the Dirichlet boundary nodes for the eikonal equation computing distances to the injection points.

In addition, all surface nodes are determined to compute distances to the next mold wall.

In [3]:
# Specify inlet location
injection_locs = [[-25.000, 111.916, 18.961]]

# Get inlet nodes
inlet_nodes = []
for injection_loc in injection_locs:
    dist_to_inlet = np.linalg.norm(points - injection_loc, axis=-1)
    inlet_nodes.append(np.argmin(dist_to_inlet, axis=0))
inlet_nodes = np.array(inlet_nodes)

In [4]:
# Get surface nodes
faces = np.vstack(
    [
        elements[:, [0, 1, 2]],
        elements[:, [1, 2, 3]],
        elements[:, [0, 2, 3]],
        elements[:, [0, 1, 3]],
    ]
)
faces = np.sort(faces, axis=1)
_, indices, counts = np.unique(faces, axis=0, return_counts=True, return_index=True)
indices = indices[np.argwhere(counts == 1)]
surface_nodes = np.unique(faces[indices]).ravel()

## Compute the distance measures

In [5]:
# Set velocity field isotropicallly to 1
f = np.stack([np.eye(3)] * len(elements))

# Compute geodesic distance to walls
fim = FIMPY.create_fim_solver(points, elements, f, device="cpu", use_active_list=False)
D_w = fim.comp_fim(surface_nodes, np.zeros_like(surface_nodes))
mesh.point_data["D_w"] = D_w


In [6]:
# Compute geodesic distance to inlet
fim = FIMPY.create_fim_solver(points, elements, f, device="cpu")
D_i = fim.comp_fim(inlet_nodes, np.zeros_like(inlet_nodes))
mesh.point_data["D_i"] = D_i

In [7]:
# Set velocity field isotropically to wall distance
D_w_elements = np.mean(D_w[elements], axis=1)
f = np.einsum("i,jk->ijk", D_w_elements, np.eye(3))

# Compute weighted distance to inlet
fim = FIMPY.create_fim_solver(points, elements, f, device="cpu")
D_i_hat = fim.comp_fim(inlet_nodes, np.zeros_like(inlet_nodes))
mesh.point_data["D_i_hat"] = D_i_hat


In [8]:
cell_coords = points[elements]
centers = np.mean(cell_coords, axis=1)

# Get node to cell relations
n2e = {k: [] for k in range(len(points))}
for i, cell in enumerate(elements):
    for j, node in enumerate(cell):
        n2e[node].append(i)

# Get cell volumes
a = cell_coords[:, 1, :] - cell_coords[:, 0, :]
b = cell_coords[:, 2, :] - cell_coords[:, 0, :]
c = cell_coords[:, 3, :] - cell_coords[:, 0, :]
omega = np.einsum("ij,ij->i", a, np.cross(b, c), optimize=True)
cell_volumes = np.abs(omega) / 6.0

# Interpolate volume field to points
node_volumes = np.array(
    [np.sum(cell_volumes[n2e[node]] / 4.0) for node in range(len(points))]
)
mesh.point_data["Volume"] = node_volumes

## Normalization
The weighted distance to the injection gate and nodal volumes are used to construct a normlized fill front in which equal increments of the fill front imply equal increemnts in filled volume. Then, the gradient of the new fill front approximation is computed with a tetragonal shape function and interpolated to nodes. 

In [9]:
order = np.argsort(D_i_hat.ravel())
ridx = np.argsort(order)
F = np.cumsum(node_volumes[order]) / np.sum(node_volumes)
fill_front_norm = F[ridx]
mesh.point_data["Normalized_fill_front"] = fill_front_norm

In [10]:
# Compute gradient
B = np.array([[-1.0, 1.0, 0.0, 0.0], [-1.0, 0.0, 1.0, 0.0], [-1.0, 0.0, 0.0, 1.0]])
J = B @ points[elements, :]
edirs = np.einsum("...ij, ...j", np.linalg.inv(J) @ B, fill_front_norm[elements])
ndirs = np.array([np.mean(edirs[n2e[node]], axis=0) for node in range(len(points))])
mesh.point_data["Fill_gradient"] = ndirs / np.linalg.norm(ndirs, axis=1)[:,None]

## Structure tensor
The structure tensor is a second order tensor measuring the directionality of the material distribution in the domain.

In [11]:
@njit(parallel=True)
def compute_structure_tensor(points, centers, volumes, indices, dist):
    num_points = len(points)
    result = np.zeros((num_points, 3, 3))
    for i in prange(num_points):
        idx = indices[i]
        diffs = centers[idx] - points[i]
        vols = volumes[idx]
        dirs = diffs.T / dist[i]
        result[i, :, :] = np.dot(vols * dirs, dirs.T) / np.sum(vols)
    return result


def tensor_sweep(points, centers, volumes, radius):
    tree = BallTree(centers)
    results = []
    chunks = np.array_split(points, 100)
    for chunk in chunks:
        n, d = tree.query_radius(chunk, r=radius, return_distance=True)
        S = compute_structure_tensor(chunk, centers, volumes, List(n), List(d))
        results.append(S)
    return np.concatenate(results)


In [12]:
# Compute structure tensor
bounding_box = np.max(points, axis=0) - np.min(points, axis=0)
radius = min(0.25 * np.max(bounding_box), np.min(bounding_box))
stensors = tensor_sweep(points, centers, cell_volumes, radius)
mesh.point_data["Structure_tensor"] = stensors.reshape((-1, 9))[:, [0, 4, 8, 1, 5, 2]]


## Export results

In [13]:
if "gmsh:geometrical" in mesh.cell_data.keys():
    mesh.cell_data.pop("gmsh:geometrical")
if "gmsh:dim_tags" in mesh.point_data.keys():
    mesh.point_data.pop("gmsh:dim_tags")
meshio.write("control_arm_hole.vtu", mesh)
